## Description
This code replicates the model discussed in the following research
> A. Chadha, R. Dara and Z. Poljak, "Convolutional Classification of Pathogenicity in H5 Avian Influenza Strains," 2019 18th IEEE International Conference On Machine Learning And Applications (ICMLA), Boca Raton, FL, USA, 2019, pp. 1570-1577.

- Within research 1202 HP sequences, 1167 LP sequences were used which were gathered from various sources such as https://www.fludb.org.  

- This code has yet to collect relevant number of data and works with only 133 HP sequences and 750 LP sequences  
- These sequences were collected from https://www.fludb.org only  
- They are all HA segments of H5 avian influenza virus of various kinds.  
- These HA segments are aligned using MUSCLE (Multiple Sequence Comparison by Log-Expectation) algorithm, available [here](https://www.fludb.org/brc/msa.spg?method=ShowCleanInputPage&decorator=influenza)  


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential, utils
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from random import shuffle

In [2]:
# Load data in records, and outputs in y
data = pd.read_csv("./H5.csv")
data = data.iloc[:2459]
records = data["HA"].apply(lambda x: np.array(list(x))).values
y = data["Pathogenicity"].values

print('Highly Pathogenic cases:', np.count_nonzero(y=="HP"))
print('Low Pathogenic cases:', np.count_nonzero(y=="LP"))

Highly Pathogenic cases: 938
Low Pathogenic cases: 1521


In [3]:
# preprocess input data
sequence = records
le_1 = LabelEncoder()
y = le_1.fit_transform(y) 
y = tf.one_hot(y, 2)
le = LabelEncoder()
seqEncoded = np.zeros((len(records),len(records[0])),dtype=int)
for i, seq in enumerate(sequence):
    seqEncoded[i] = le.fit_transform(seq)

oneHotSeq = tf.one_hot(seqEncoded, depth=21).numpy()

# remove alignment character's one hot
for seq in oneHotSeq:
    for protein in seq:
        if protein[0] == 1:
            protein[0] = 0
print('(samples, proteins, one-hot-encoding) ::',oneHotSeq.shape)
print(oneHotSeq[:1],y)
trainTestValues=[]
noOfEntries=oneHotSeq.shape[0]
for i in range(noOfEntries):
  itemArray=[oneHotSeq[i],y[i]]
  trainTestValues.append(itemArray)
shuffle(trainTestValues)
testSplitIndex=int(noOfEntries*0.75)
train=trainTestValues[:testSplitIndex]
test=trainTestValues[testSplitIndex:]
X_train=[i[0] for i in train]
X_train=np.array(X_train)
print(X_train[0])
y_train=[i[1] for i in train]
X_test=[i[0] for i in test]
y_test=[i[1] for i in test]
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
print(len(X_train)+len(X_test),noOfEntries)
#X_train,y_train,X_test,y_test = train_test_split(oneHotSeq,y,test_size=0.25)

(samples, proteins, one-hot-encoding) :: (2459, 589, 21)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]] tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]], shape=(2459, 2), dtype=float32)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
1844
1844
615
615
2459 2459


In [4]:
# building Model
model = Sequential()

model.add(Conv1D(20,
                 kernel_size=2,
                 strides=2,
                 activation='relu',
                 input_shape=(oneHotSeq.shape[1], oneHotSeq.shape[2],)))
model.add(MaxPool1D(pool_size=2,
                    strides=2,
                    padding='valid'))
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 294, 20)           860       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 147, 20)           0         
_________________________________________________________________
flatten (Flatten)            (None, 2940)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 5882      
Total params: 6,742
Trainable params: 6,742
Non-trainable params: 0
_________________________________________________________________


In [5]:
# compiling model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# compiling model
'''optimizer = RMSprop(learning_rate=0.001, rho=0.9)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])'''

"optimizer = RMSprop(learning_rate=0.001, rho=0.9)\nmodel.compile(loss='binary_crossentropy',\n              optimizer=optimizer,\n              metrics=['accuracy'])"

In [6]:
# training model
# Due to lack of data model is underfitting (training loss >> validation loss)
class_weights = {0: 1.,
                 1: 5}
'''for st in range(100):
  class_weight[1]=2.5+st*0.01
  print(class_weight)
  model.fit(
    np.array(X_train), np.array(y),
    batch_size=32,
    epochs=1,
    validation_split=0.3,
    shuffle=True,
    class_weight=class_weight
    )'''
model.fit(
  np.array(X_train), np.array(y_train),
  batch_size=100,
  epochs=13,
  validation_split=0.2,
  shuffle=True,
  class_weight=class_weights
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 1475 samples, validate on 369 samples
Epoch 1/13
1475/1475 [==============================] - 1s 489us/sample - loss: 1.5067 - accuracy: 0.6075 - val_loss: 1.0672 - val_accuracy: 0.6314
Epoch 2/13
1475/1475 [==============================] - 0s 168us/sample - loss: 0.9093 - accuracy: 0.6203 - val_loss: 0.7189 - val_accuracy: 0.6314
Epoch 3/13
1475/1475 [==============================] - 0s 173us/sample - loss: 0.6280 - accuracy: 0.6610 - val_loss: 0.5238 - val_accuracy: 0.9512
Epoch 4/13
1475/1475 [==============================] - 0s 150us/sample - loss: 0.4524 - accuracy: 0.9546 - val_loss: 0.3863 - val_accuracy: 0.9512
Epoch 5/13
1475/1475 [==============================] - 0s 150us/sample - loss: 0.3432 - accuracy: 0.9532 - val_loss: 0.3078 - val_accuracy: 0.9512
Epoch 6/13
1475/1475 [==============================] - 0s 160us/sample - loss: 0.2835 - accuracy: 0.9546 - val_loss: 0.2649 - val_accuracy: 0.9512
Epoch 7/13
1475

In [7]:
y_pred = model.predict(np.array(X_test))
y_pred=y_pred>0.5
print(classification_report(np.array(y_test),y_pred))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94       237
           1       0.94      1.00      0.97       378

   micro avg       0.96      0.96      0.96       615
   macro avg       0.97      0.95      0.95       615
weighted avg       0.96      0.96      0.96       615
 samples avg       0.96      0.96      0.96       615

